In [20]:
import pandas as pd
import numpy as np
import os,sys
import itertools
import json
import shutil
import multiprocessing as mp
from multiprocessing import Pool
cpus = int(np.floor((mp.cpu_count()-1) /2))
print(cpus)

35


In [2]:
os.chdir('/data/jake/gene-fusion/python')

Tumour sample filtered ped file. Processed from `prostate_stix_ped.ipynb`

In [8]:
df = pd.read_csv('../data/meta/icgc-GRCh37-prostate_cancer.ped',sep='\t').sort_values('sample').reset_index(drop=True)
print(df.shape)
df.head()

(142, 20)


,sample,alt_file,objectid,file_name,icgc_donor,icgc_specimen_id,specimen_type,icgc_sample_id,project,study,data_type,experimental_strategy,format,analysisid,ega_dataset_id,ega_analysis_id,ega_file_id,pcawg,location,label
0,FI10093,FI10093.excord.bed.gz,c1b0f8bd-b2ad-5d03-84cd-942971bf193a,3bcb0039e46a4ca217d29b0c77a1ef7e.bam,DO51133,SP112953,Primary tumour - solid tissue,SA530604,PRAD-CA,PCAWG,BAM,WGS,BAM,b513c61d-ff71-4ddc-9888-1a406942069e,EGAD00001002128,EGAZ00001225228,EGAF00001162820,whitelist,EGA,cancer
1,FI10766,FI10766.excord.bed.gz,c2ec5d59-5ead-5792-aacb-80078ceff799,7a0090e6dabe350ecc19afb773d430e7.bam,DO51136,SP112959,Primary tumour - solid tissue,SA530610,PRAD-CA,PCAWG,BAM,WGS,BAM,b9b072b8-59fd-4d39-a538-03c05b907584,EGAD00001002128,EGAZ00001225318,EGAF00001163090,whitelist,EGA,cancer
2,FI10784,FI10784.excord.bed.gz,fb4faccb-cbcd-5706-bb5a-a8180c7f22bd,2238ab07083b519abceb375f12d7f5f9.bam,DO51067,SP112821,Primary tumour - solid tissue,SA530479,PRAD-CA,PCAWG,BAM,WGS,BAM,e2dd4dfe-980c-4587-bde6-7e9f466d5848,EGAD00001002128,EGAZ00001223260,EGAF00001084921,whitelist,EGA,cancer
3,FI11979,FI11979.excord.bed.gz,fd22f0c2-d50f-551c-9b11-5496df8a56ef,1c2dad03e47d34118d63bf4cb89d97a4.bam,DO51142,SP112971,Primary tumour - solid tissue,SA530622,PRAD-CA,PCAWG,BAM,WGS,BAM,2b5fe918-e0f8-4c1f-8024-ddcd55557b8d,EGAD00001002128,EGAZ00001225221,EGAF00001162799,whitelist,EGA,cancer
4,FI12251,FI12251.excord.bed.gz,33623629-be74-5373-8457-0e43f805b935,f1c61be3f2beb330e3aa731d8c325ecb.bam,DO51159,SP113005,Primary tumour - solid tissue,SA530656,PRAD-CA,PCAWG,BAM,WGS,BAM,64e6a38c-cf95-4264-b039-99a34dfe3d1a,EGAD00001002128,EGAZ00001225184,EGAF00001160759,whitelist,EGA,cancer


In [10]:
df.specimen_type.value_counts()

specimen_type
Primary tumour - solid tissue                         107
Metastatic tumour - metastasis to distant location     20
Metastatic tumour - lymph node                         15
Name: count, dtype: int64

In [13]:
k=12
shards = np.array_split(df['alt_file'].to_list(),k)
shards[0:2]

[array(['FI10093.excord.bed.gz', 'FI10766.excord.bed.gz',
        'FI10784.excord.bed.gz', 'FI11979.excord.bed.gz',
        'FI12251.excord.bed.gz', 'FI12844.excord.bed.gz',
        'FI12977.excord.bed.gz', 'FI13089.excord.bed.gz',
        'FI13504.excord.bed.gz', 'FI14823.excord.bed.gz',
        'FI14824.excord.bed.gz', 'FI15561.excord.bed.gz'], dtype='<U22'),
 array(['FI16501.excord.bed.gz', 'FI16518.excord.bed.gz',
        'FI16535.excord.bed.gz', 'FI16552.excord.bed.gz',
        'FI18299.excord.bed.gz', 'FI19096.excord.bed.gz',
        'FI19114.excord.bed.gz', 'FI2021.excord.bed.gz',
        'FI2039.excord.bed.gz', 'FI21896.excord.bed.gz',
        'FI22097.excord.bed.gz', 'FI22421.excord.bed.gz'], dtype='<U22')]

In [25]:
# shard files
dir_prostate='/data/jake/gene-fusion/data/prostate'
shard_meta = { i: k.tolist() for i,k in enumerate(shards)}
with open(os.path.join(dir_prostate, 'shard.json'), 'w') as f:
    json.dump(shard_meta, f, indent=4)

In [26]:
prefix_src='/data/jake/gene-fusion/data/cancer_data/prostate/prostate_sort'
#dir_cp='/data/jake/gene-fusion/data/prostate/copies'
dir_shards='/data/jake/gene-fusion/data/prostate/shards'
# top-level
#if not os.path.exists(dir_cp):
    #os.mkdir(dir_cp)
if not os.path.exists(dir_shards):
    os.mkdir(dir_shards)
# copies
for i in range(k):
    d_shard = os.path.join(dir_shards,f'shard_{i}')
    d_beds = os.path.join(d_shard,'beds')
    if not os.path.exists(d_shard):
        os.mkdir(d_shard)
    if not os.path.exists(d_beds):
        os.mkdir(d_beds)

In [27]:
data = []
for shard in shard_meta.keys():
    for file in shard_meta[shard]:
        src = os.path.join(prefix_src, file)
        dst = os.path.join(dir_shards, f'shard_{shard}', 'beds', file)
        data.append((src,dst))
        #print(src, dst)
        # if not os.path.exists(dst):
        #     shutil.copy(src, dst)


In [29]:
with Pool(processes=cpus) as pool:
        results = pool.starmap(shutil.copy, data)

In [ ]:
#giggle index -i "beds/*gz" -o repeat_sort_b -f -s